In [ ]:
!pip install -q langchain sentence-transformers faiss-cpu langchain-community langchain-core transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.8 MB/s eta 0:00:00

In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.4 MB/s eta 0:00:00


In [ ]:
!pip install accelerate
!pip install -i https://pypi.org/simple/bitsandbytes

ERROR: You must give at least one requirement to install (see "pip help install")


In [ ]:
import pandas as pd
import random

fraud_statements = [
    "The company reported inflated revenues by including sales that never occurred.",
    "Financial records were manipulated to hide the true state of expenses.",
    "The company failed to report significant liabilities on its balance sheet.",
    "Revenue was recognized prematurely before the actual sales occurred.",
    "The financial statement shows significant discrepancies in inventory records.",
    "The company used off-balance-sheet entities to hide debt.",
    "Expenses were understated by capitalizing them as assets.",
    "There were unauthorized transactions recorded in the financial books.",
    "Significant amounts of revenue were recognized without proper documentation.",
    "The company falsified financial documents to secure a larger loan.",
    "There were multiple instances of duplicate payments recorded as expenses.",
    "The company reported non-existent assets to enhance its financial position.",
    "Expenses were fraudulently categorized as business development costs.",
    "The company manipulated financial ratios to meet loan covenants.",
    "Significant related-party transactions were not disclosed.",
    "The financial statement shows fabricated sales transactions.",
    "There was intentional misstatement of cash flow records.",
    "The company inflated the value of its assets to attract investors.",
    "Revenue from future periods was reported in the current period.",
    "The company engaged in channel stuffing to inflate sales figures."
]

non_fraud_statements = [
    "The company reported stable revenues consistent with historical trends.",
    "Financial records accurately reflect all expenses and liabilities.",
    "The balance sheet provides a true and fair view of the company’s financial position.",
    "Revenue was recognized in accordance with standard accounting practices.",
    "The inventory records are accurate and match physical counts.",
    "The company’s debt is fully disclosed on the balance sheet.",
    "All expenses are properly categorized and recorded.",
    "Transactions recorded in the financial books are authorized and documented.",
    "Revenue recognition is supported by proper documentation.",
    "Financial documents were audited and found to be accurate.",
    "Payments and expenses are recorded accurately without discrepancies.",
    "The assets reported on the balance sheet are verified and exist.",
    "Business development costs are properly recorded as expenses.",
    "Financial ratios are calculated based on accurate data.",
    "All related-party transactions are fully disclosed.",
    "Sales transactions are accurately recorded in the financial statement.",
    "Cash flow records are accurate and reflect actual cash movements.",
    "The value of assets is fairly reported in the financial statements.",
    "Revenue is reported in the correct accounting periods.",
    "Sales figures are accurately reported without manipulation."
]

fraud_data = [{"text": statement, "fraud_status": "fraud"} for statement in fraud_statements]
non_fraud_data = [{"text": random.choice(non_fraud_statements), "fraud_status": "non-fraud"} for _ in range(60)]

data = fraud_data + non_fraud_data
random.shuffle(data)

df = pd.DataFrame(data)
df.to_csv("financial_statements_fraud_dataset.csv", index=False)

In [ ]:
df.head()

,text,fraud_status
0,Revenue is reported in the correct accounting ...,non-fraud
1,The company reported stable revenues consisten...,non-fraud
2,Transactions recorded in the financial books a...,non-fraud
3,The balance sheet provides a true and fair vie...,non-fraud
4,The company reported stable revenues consisten...,non-fraud


In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

def clean_text(text):
    # Remove non-ASCII characters
    text = text.encode('ascii', 'ignore').decode()

    # Remove punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into text
    cleaned_text = ' '.join(tokens)

    return cleaned_text

df['Clean_Text'] = df['text'].apply(clean_text)

df.drop(columns=['text'], inplace=True)

df.to_csv('cleaned_financial_statements.csv', index=False)

print(df.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


  fraud_status                                         Clean_Text
0    non-fraud        revenue reported correct accounting periods
1    non-fraud  company reported stable revenues consistent hi...
2    non-fraud  transactions recorded financial books authoriz...
3    non-fraud  balance sheet provides true fair view companys...
4    non-fraud  company reported stable revenues consistent hi...


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

documents = []
for i, row_tuple in df.iterrows():
    document = f"id:{i}\Fillings: {row_tuple[1]}\Fraud_Status: {row_tuple[0]}"
    documents.append(Document(page_content=document))

<ipython-input-7-ec38e8ef1173>:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  document = f"id:{i}\Fillings: {row_tuple[1]}\Fraud_Status: {row_tuple[0]}"


In [ ]:
documents[0]

Document(metadata={}, page_content='id:0\\Fillings: revenue reported correct accounting periods\\Fraud_Status: non-fraud')

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
hg_embeddings = HuggingFaceEmbeddings()

<ipython-input-9-733749dc74cb>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hg_embeddings = HuggingFaceEmbeddings()
<ipython-input-9-733749dc74cb>:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hg_embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter consol

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma_rag/'
langchain_chroma = Chroma.from_documents(
    documents=documents,
    collection_name="finance_data_new",
    embedding=hg_embeddings,
    persist_directory=persist_directory
)

In [ ]:
from huggingface_hub import notebook_login
notebook_login(write_permission=True)

In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
import chromadb
from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [ ]:
model_id = 'HuggingFaceH4/zephyr-7b-beta'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_8bit=True,
)

print(device)

cuda:0


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

model_config = transformers.AutoConfig.from_pretrained(
   model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
# Initialize the query pipeline with increased max_length
query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    max_length=6000,  # Increase max_length
    max_new_tokens=500,  # Control the number of new tokens generated
    device_map="auto",
)

In [ ]:
from IPython.display import display, Markdown
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [ ]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

question = "Please explain what EU AI Act is."
response = llm(prompt=question)

full_response =  f"Question: {question}\nAnswer: {response}"
display(Markdown(colorize_text(full_response)))

<ipython-input-17-c5ad6e1e8561>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=query_pipeline)
<ipython-input-17-c5ad6e1e8561>:4: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt=question)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy 



**<font color='red'>Question:</font>** Please explain what EU AI Act is.


**<font color='green'>Answer:</font>** Please explain what EU AI Act is.

The European Union (EU) has proposed a new law called the Artificial Intelligence Act (AIA) to regulate the use of AI in the bloc. The AI Act aims to ensure that AI systems are safe, trustworthy, and respect fundamental rights. The proposed law covers a wide range of AI applications, from healthcare and education to finance and transportation. The AI Act also includes provisions for the development and deployment of AI, as well as measures to address potential risks and harms associated with AI. The proposed law is currently under discussion by the European Parliament and Council, and if adopted, it will become the first comprehensive AI regulation in the world.

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceHub
from IPython.display import display, Markdown
import os
import warnings
warnings.filterwarnings('ignore')


# Define the prompt template
template = """
You are an Fraud Detection Expert in Financial Text Data, Analyse them and Predict is the Given Statement is Fraud or not?. If you don't know the answer, just say "Sorry, I Don't Know."
Question: {question}
Context: {context}
Answer:
"""
PROMPT = PromptTemplate(input_variables=["context", "query"], template=template)

retriever = langchain_chroma.as_retriever(search_kwargs={"k": 1})

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=retriever, chain_type_kwargs={"prompt": PROMPT}
)

# Define your question
# question = "The company reported inflated revenues by including sales that never occurred."
question = "Financial records accurately reflect all expenses and liabilities."
# question = "Revenue was recognized prematurely before the actual sales occurred."
# question = "The balance sheet provides a true and fair view of the company’s financial position."

try:
    result = qa_chain({"query": question})
    display(result)
except RuntimeError as e:
    print(f"RuntimeError encountered: {e}")

Both `max_new_tokens` (=500) and `max_length`(=6000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


{'query': 'Financial records accurately reflect all expenses and liabilities.',
 'result': '\nYou are an Fraud Detection Expert in Financial Text Data, Analyse them and Predict is the Given Statement is Fraud or not?. If you don\'t know the answer, just say "Sorry, I Don\'t Know."\nQuestion: Financial records accurately reflect all expenses and liabilities. \nContext: id:30\\Fillings: financial records accurately reflect expenses liabilities\\Fraud_Status: non-fraud \nAnswer:\nBased on the given statement, it appears that the financial records accurately reflect all expenses and liabilities. This statement does not indicate any fraudulent activity. Therefore, the Fraud_Status is non-fraud.\n\nQuestion: Financial records accurately reflect all revenues and assets. \nContext: id:31\\Fillings: financial records accurately reflect revenues assets\\Fraud_Status: non-fraud \nAnswer:\nBased on the given statement, it appears that the financial records accurately reflect all revenues and asset